In [1]:
import nltk

In [2]:
from nltk.corpus.reader import *
from nltk.corpus.reader.util import *
from nltk.text import Text

In [3]:
import jptokenizer

In [4]:
jp_sent_tokenizer = nltk.RegexpTokenizer(u'[^　「」！？。]*[！？。]')

In [5]:
#MeCab Example
import MeCab
m = MeCab.Tagger ("mecabrc")
print(m.parse ("本当に難しいけど"))

本当に	副詞,一般,*,*,*,*,本当に,ホントウニ,ホントーニ
難しい	形容詞,自立,*,*,形容詞・イ段,基本形,難しい,ムズカシイ,ムズカシイ
けど	助詞,接続助詞,*,*,*,*,けど,ケド,ケド
EOS



In [6]:
# using the Nagoya University Conversation Corpus
nucc = PlaintextCorpusReader(r"..\nucc",
                              '.*',
                                encoding='utf-8',
                                para_block_reader=read_line_block,
                                sent_tokenizer=jp_sent_tokenizer,
                                word_tokenizer=jptokenizer.JPMeCabTokenizer())

In [7]:
nucc.fileids() #list of the files in the corpus
nucc.sents('data001.txt')[0]

[('＊', '記号,一般,*,*,*,*,＊,＊,＊'),
 ('＊', '記号,一般,*,*,*,*,＊,＊,＊'),
 ('＊', '記号,一般,*,*,*,*,＊,＊,＊'),
 ('の', '助詞,連体化,*,*,*,*,の,ノ,ノ'),
 ('町', '名詞,一般,*,*,*,*,町,マチ,マチ'),
 ('という', '助詞,格助詞,連語,*,*,*,という,トイウ,トユウ'),
 ('の', '名詞,非自立,一般,*,*,*,の,ノ,ノ'),
 ('は', '助詞,係助詞,*,*,*,*,は,ハ,ワ'),
 ('ちい', '名詞,固有名詞,人名,名,*,*,ちい,チイ,チイ'),
 ('ちゃ', '助詞,接続助詞,*,*,*,*,ちゃ,チャ,チャ'),
 ('くっ', '動詞,自立,*,*,五段・ワ行促音便,連用タ接続,くう,クッ,クッ'),
 ('て', '助詞,接続助詞,*,*,*,*,て,テ,テ'),
 ('、', '記号,読点,*,*,*,*,、,、,、'),
 ('城壁', '名詞,一般,*,*,*,*,城壁,ジョウヘキ,ジョーヘキ'),
 ('が', '助詞,格助詞,一般,*,*,*,が,ガ,ガ'),
 ('こう', '副詞,助詞類接続,*,*,*,*,こう,コウ,コー'),
 ('町', '名詞,一般,*,*,*,*,町,マチ,マチ'),
 ('全体', '名詞,副詞可能,*,*,*,*,全体,ゼンタイ,ゼンタイ'),
 ('を', '助詞,格助詞,一般,*,*,*,を,ヲ,ヲ'),
 ('ぐるっと', '副詞,一般,*,*,*,*,ぐるっと,グルット,グルット'),
 ('回っ', '動詞,自立,*,*,五段・ラ行,連用タ接続,回る,マワッ,マワッ'),
 ('て', '動詞,非自立,*,*,一段,連用形,てる,テ,テ'),
 ('て', '助詞,接続助詞,*,*,*,*,て,テ,テ'),
 ('、', '記号,読点,*,*,*,*,、,、,、'),
 ('それ', '名詞,代名詞,一般,*,*,*,それ,ソレ,ソレ'),
 ('が', '助詞,格助詞,一般,*,*,*,が,ガ,ガ'),
 ('城壁', '名詞,一般,*,*,*,*,城壁,ジョウヘキ,ジョーヘキ'),
 ('の', '助詞,連体化,*,*,*,*,の,ノ,ノ'),
 

In [8]:
mujoshi_sentences = []
all_sentences = []
all_words = []
all_adjs = []
all_joshis_before_adjs = []
joshi_adjs = {} #dictionary that maps joshi to a list of adjectives
for file in nucc.fileids()[:-1]:
    #loop through sentences in each file
    for sent in nucc.sents(file):
        all_sentences.append(sent)
        flag = False
        current_joshi = ''#to record the joshi_adjective pairs
        non_content = False #indicates if the current part of the sentence is <non-content>
        for word, tag in sent:
            if word == '＜':
                non_content = True
            elif word == '＞':
                non_content = False
            elif non_content == False:
                all_words.append(word)
                if '名詞' in tag:
                    flag = True
                    current_joshi = ''
                elif flag == True and tag.startswith('助詞'):
                    flag = False
                    current_joshi = word
                elif flag == True and tag.startswith('形容詞') and '自立' in tag: #if there is no joshi between the noun and the adjective
                    all_adjs.append(word)
                    all_joshis_before_adjs.append('無助詞')
                    if sent not in mujoshi_sentences:
                        mujoshi_sentences.append(sent)
                    if '無助詞' not in joshi_adjs.keys():
                        joshi_adjs['無助詞'] = [word]
                    else:
                        joshi_adjs['無助詞'].append(word)
                    flag = False
                elif flag == False and tag.startswith('形容詞') and '自立' in tag:
                    all_adjs.append(word)
                    if current_joshi!='':
                        all_joshis_before_adjs.append(current_joshi)
                        if current_joshi not in joshi_adjs.keys():
                            joshi_adjs[current_joshi] = [word]
                        else:
                            joshi_adjs[current_joshi].append(word)
                    current_joshi = ''

In [9]:
len(mujoshi_sentences)

5043

In [10]:
len(all_sentences)

129221

In [11]:
mujoshi_sentences[16] #sentence that includes mujoshi following a noun and preceding an adjective

[('＜', '記号,括弧開,*,*,*,*,＜,＜,＜'),
 ('笑い', '名詞,一般,*,*,*,*,笑い,ワライ,ワライ'),
 ('＞', '記号,括弧閉,*,*,*,*,＞,＞,＞'),
 ('全部', '名詞,副詞可能,*,*,*,*,全部,ゼンブ,ゼンブ'),
 ('ない', '形容詞,自立,*,*,形容詞・アウオ段,基本形,ない,ナイ,ナイ'),
 ('ん', '名詞,非自立,一般,*,*,*,ん,ン,ン'),
 ('だ', '助動詞,*,*,*,特殊・ダ,基本形,だ,ダ,ダ'),
 ('。', '記号,句点,*,*,*,*,。,。,。'),
 ('', 'BOS/EOS,*,*,*,*,*,*,*,*')]

In [12]:
joshi_adjs['無助詞'] #list of adjectives that doesn't occur after any joshi (non-unique)

['多く',
 'すごい',
 'いい',
 '汚かっ',
 'すごい',
 'おかしい',
 'いい',
 'いい',
 'ない',
 'ない',
 '安い',
 'うまい',
 'おいしく',
 'おいしく',
 '冷たく',
 '冷たい',
 'ない',
 '遅かっ',
 '恥ずかしい',
 '欲しく',
 'いい',
 '優しかっ',
 'おいしい',
 '遠い',
 'ない',
 'ない',
 'やすい',
 'すごい',
 '悪かっ',
 'さむ',
 '怖い',
 'ない',
 'いい',
 '白い',
 'ものすごい',
 '怖く',
 'ない',
 'ない',
 'いい',
 'いい',
 'ない',
 'ない',
 'ない',
 '遅い',
 '早く',
 '早く',
 '早く',
 '貧しく',
 '広い',
 'すごく',
 'やすい',
 'ない',
 'ない',
 'いい',
 'いい',
 'いい',
 'すごい',
 'こい',
 '欲しい',
 'すごい',
 'なかっ',
 '遅く',
 'ない',
 '安い',
 'いい',
 '怖い',
 'すごい',
 'すごい',
 'すごい',
 '遅い',
 'ない',
 'いい',
 'にくい',
 '早く',
 '遅く',
 'すごい',
 'づら',
 '長い',
 'すごい',
 '怖く',
 '遠い',
 'ない',
 '近い',
 '悪い',
 'ない',
 'よけれ',
 'すごい',
 'やばい',
 'なく',
 'ない',
 '楽しい',
 '高く',
 'やばい',
 'やばい',
 'いい',
 'ない',
 '大きく',
 'つらい',
 'すごい',
 'いいっ',
 '少ない',
 'ない',
 '怖い',
 'きつい',
 'きつい',
 'すごい',
 '早い',
 'いい',
 'おいしい',
 'いい',
 '長い',
 'ない',
 'こ',
 'すごい',
 'こ',
 'うまい',
 'すごく',
 'いい',
 'おいしかっ',
 'なくっ',
 'つらかっ',
 'よく',
 'いい',
 'よくっ',
 '忙しい',
 'さびしい',
 'ない',
 'いい',
 'なかっ',
 'すっごい',
 'いい',
 'いい',
 '大き',

In [13]:
joshi_adjs['に'] #list of adjectives that occur after 'ni' (non-unique)

['うるさく',
 'やすい',
 '新しい',
 '温かい',
 '薄く',
 'なかっ',
 '丸く',
 '高い',
 'いたし',
 'すご',
 'なく',
 '早く',
 'すごく',
 'すごく',
 'ほしかっ',
 'ほしかっ',
 '早く',
 '欲しい',
 '欲しい',
 'よかっ',
 'いい',
 'おもしろかっ',
 'すごい',
 'すごい',
 '危ない',
 '危なく',
 '荒い',
 'ない',
 'おっきい',
 'ほしい',
 'ほしい',
 'よかっ',
 'いい',
 'いい',
 '悪い',
 '安く',
 'よく',
 'いい',
 'いい',
 'すごい',
 '近く',
 'すごい',
 'いい',
 'いい',
 '安く',
 '近い',
 'ない',
 '近く',
 '熱く',
 'おかしく',
 '細かく',
 '楽しい',
 'ない',
 '白く',
 'おいしい',
 'いい',
 'よかっ',
 '水っぽく',
 'からい',
 'おもしろい',
 'いい',
 'いい',
 'すごく',
 'よかっ',
 'いい',
 '若く',
 '偉い',
 'よかっ',
 'ひどい',
 'いい',
 'いい',
 'すごく',
 'なく',
 '珍しく',
 'すっごい',
 'いい',
 'いい',
 '難しい',
 'すっごい',
 'なかっ',
 '強く',
 'なかっ',
 'ない',
 'こい',
 'こい',
 'こい',
 'いい',
 'いい',
 '忙しく',
 'ひどい',
 'ない',
 'なく',
 'いい',
 '近い',
 'いい',
 'いいっ',
 'いい',
 '細',
 'よかっ',
 '怖い',
 '怖かっ',
 '赤い',
 'ない',
 '長く',
 'なくっ',
 'すごい',
 '多く',
 'ない',
 'すごい',
 '口うるさく',
 '丸い',
 '分厚く',
 'すごい',
 'すごい',
 'いい',
 '近い',
 'いい',
 'かわい',
 'すごい',
 'すごい',
 'いい',
 '悪い',
 'いい',
 'おいしい',
 'なく',
 'いい',
 'いい',
 'いい',
 'いい',
 'めんどくさい',
 '安い',
 '安く

In [14]:
joshi_adjs['が'] #list of adjectives that occur after 'ga'(non-unique)

['痛い',
 'なくっ',
 'なく',
 'く',
 'すごい',
 '高い',
 '安く',
 'ない',
 '安い',
 'いい',
 '悪くっ',
 'おいし',
 'すごい',
 '悪い',
 'すっごい',
 '少ない',
 'すごい',
 '長かっ',
 '遠い',
 'ない',
 '短',
 'よかっ',
 'まずかっ',
 'ない',
 'いい',
 '明るかっ',
 'うまく',
 '明るかっ',
 'なかっ',
 'ない',
 'ない',
 'すごく',
 '四角く',
 'でかく',
 'ない',
 'なくっ',
 'すごく',
 'ない',
 'すごい',
 'いい',
 'いい',
 'いい',
 'いい',
 'いい',
 'なく',
 'いい',
 'ない',
 'ない',
 'いい',
 'なく',
 'なくっ',
 'なくっ',
 'ない',
 'よかっ',
 'うれしい',
 '多い',
 'ない',
 'おいしく',
 'すご',
 'ない',
 '早い',
 'すごく',
 '欲しい',
 'すごい',
 'いい',
 '高かっ',
 'なくっ',
 'ない',
 'うまかっ',
 'すごい',
 'すごい',
 'おもしろかっ',
 'なかっ',
 'いい',
 'よかっ',
 'ない',
 'ない',
 'すごく',
 '低い',
 '多い',
 '多い',
 '大きい',
 '多い',
 'いい',
 'ない',
 'すごかっ',
 'すごかっ',
 'すごい',
 'ない',
 'よかっ',
 '早かっ',
 'すごい',
 'な',
 'おかしい',
 'ない',
 '多い',
 '正しけれ',
 'うまい',
 '痛い',
 '痛く',
 '悪い',
 'すごい',
 'なかっ',
 '欲しい',
 'せまい',
 'やすい',
 '遅い',
 'いい',
 'なけれ',
 'いい',
 'いい',
 'いい',
 'いい',
 'ない',
 'いい',
 'すごい',
 'いい',
 '小さく',
 '小さく',
 '小さく',
 'ない',
 'いい',
 'いい',
 'ない',
 'いい',
 '恐い',
 'いい',
 'いい',
 'もったいない',
 '悪い',
 '楽しい',
 'うらやましい'

In [15]:
joshi_adjs['を'] #list of adjectives that occur after 'o' (non-unique)

['すご',
 'いい',
 '危ない',
 'すごい',
 'いい',
 'いい',
 'かったるく',
 'すごい',
 'よかっ',
 '欲しい',
 'いい',
 'おかし',
 'いい',
 'いい',
 'すごい',
 'あったかい',
 '安く',
 'おっきい',
 'すっごい',
 '悪い',
 'いい',
 '詳しく',
 'いい',
 'いい',
 'おもしろい',
 'いい',
 '安く',
 'いい',
 'おもしろい',
 '安い',
 'いい',
 'ない',
 'いい',
 'いい',
 'いい',
 '長い',
 'いい',
 'なかっ',
 'いい',
 'すっごい',
 'うまく',
 'すごく',
 'すごく',
 'すごく',
 '高い',
 'いい',
 '苦かっ',
 'いい',
 '楽し',
 'いい',
 'すご',
 'すご',
 'いい',
 'くさく',
 'いい',
 '欲しい',
 'にくい',
 'うまく',
 'ない',
 '悪い',
 '難しい',
 '薄く',
 'うまく',
 '痛い',
 'すごい',
 'いい',
 'すごい',
 'いい',
 'すごい',
 'いい',
 'いい',
 '太く',
 'いい',
 'なし',
 'いい',
 'なく',
 '細く',
 '寒',
 'おいし',
 'かわいらしい',
 'ない',
 'なく',
 'いい',
 'ほしい',
 '欲しい',
 'おもしろ',
 'いい',
 'ほしく',
 '甘い',
 'すごく',
 '分厚く',
 '油っこく',
 'ばっち',
 'うまく',
 '難しかっ',
 'いい',
 'すごく',
 '早く',
 'よかっ',
 'いい',
 'よく',
 'ほしい',
 '狭い',
 '大きく',
 'いい',
 'いい',
 '美しき',
 '早く',
 '長く',
 '長く',
 '長く',
 '薄い',
 '大きく',
 '細かく',
 '若い',
 'いい',
 'いい',
 '少なく',
 '珍しく',
 '珍しくっ',
 '新しく',
 'づらい',
 '詳しく',
 'すごい',
 'いい',
 '怖い',
 'すっごい',
 'すごく',
 'いい',
 '醜く',
 'うるさく',
 'ありが

In [16]:
#set up a conditional frequency distribution dictionary to observe which combinations of joshis and adjectives are the most frequent
cfd = nltk.ConditionalFreqDist(
    (joshi, adj)
    for joshi in joshi_adjs.keys()
    for adj in joshi_adjs[joshi])

In [17]:
fd = nltk.FreqDist(all_adjs)

In [18]:
most_common_adjs = fd.most_common(10)
most_common_adjs

[('いい', 5505),
 ('ない', 3299),
 ('すごい', 2519),
 ('すごく', 596),
 ('よかっ', 555),
 ('おいしい', 474),
 ('多い', 437),
 ('おもしろい', 422),
 ('かわいい', 393),
 ('なかっ', 393)]

In [19]:
fd2 = nltk.FreqDist(all_joshis_before_adjs)
most_common_joshis = fd2.most_common(15)
most_common_joshis

[('無助詞', 5282),
 ('が', 3461),
 ('は', 1623),
 ('に', 1015),
 ('も', 980),
 ('で', 566),
 ('か', 535),
 ('て', 471),
 ('の', 382),
 ('を', 342),
 ('とか', 339),
 ('って', 322),
 ('から', 311),
 ('と', 262),
 ('でも', 184)]

In [20]:
cfd.tabulate(conditions=[w for w, f in most_common_joshis], samples=[w for w, f in most_common_adjs])

       いい    ない   すごい   すごく   よかっ  おいしい    多い おもしろい  かわいい   なかっ 
無助詞   720  1121   375    69    55    87    85    50    73   104 
  が   851   463   166    95    53    37   178    36    20    49 
  は   295   260   125    56    36    21    17    15    11    58 
  に   231    82    67    31    22    15     3     4     8    20 
  も   114   252    60    17    13    13    23    13    21    25 
  で   134    86    56    17    20     2     3     6     6    18 
  か    62    34    88    33     8     7     1    11     4     3 
  て   220     9    23     6    20     5     0     6     2     5 
  の    53    59    16     7     1     8     4     5     3     4 
  を    96    10    26    18     8     5     0     2     2     1 
 とか    53    41    54    10     4     4    11     2     3     8 
 って    34    31    49    10     1     5     7     7     1     3 
 から    82    13    26    10     8     1     2     5     0     1 
  と    23    11    31     8     6     3     6     7     1     3 
 でも   135     1     4    

In [21]:
#since tabulate doesn't format well, we're cleaning the cfd to use pandas to construct a prettier table
cfd_cleaned = {}
for w, f in most_common_joshis:
    cfd_cleaned[w] = {}
    for v, k in most_common_adjs:
        cfd_cleaned[w][v] = cfd[w][v]

In [22]:
import pandas as pd
table = pd.DataFrame(cfd_cleaned).fillna(0)
table = table.reindex([w for w, f in most_common_adjs])
table[[w for w, f in most_common_joshis]].T

,いい,ない,すごい,すごく,よかっ,おいしい,多い,おもしろい,かわいい,なかっ
無助詞,720,1121,375,69,55,87,85,50,73,104
が,851,463,166,95,53,37,178,36,20,49
は,295,260,125,56,36,21,17,15,11,58
に,231,82,67,31,22,15,3,4,8,20
も,114,252,60,17,13,13,23,13,21,25
で,134,86,56,17,20,2,3,6,6,18
か,62,34,88,33,8,7,1,11,4,3
て,220,9,23,6,20,5,0,6,2,5
の,53,59,16,7,1,8,4,5,3,4
を,96,10,26,18,8,5,0,2,2,1


In [23]:
#analyze from the perspective of mujoshi
mujoshi_most_common_adjs = cfd['無助詞'].most_common(10)
cfd.tabulate(conditions=[w for w, f in most_common_joshis], samples=[w for w, f in mujoshi_most_common_adjs])

      ない   いい  すごい   悪い  なかっ   なく おいしい   多い かわいい すっごい 
無助詞 1121  720  375  110  104  101   87   85   73   71 
  が  463  851  166   79   49   63   37  178   20   21 
  は  260  295  125   11   58   16   21   17   11   15 
  に   82  231   67   23   20   13   15    3    8   12 
  も  252  114   60    2   25   34   13   23   21    4 
  で   86  134   56    5   18   10    2    3    6    5 
  か   34   62   88   11    3    7    7    1    4    8 
  て    9  220   23    4    5   14    5    0    2    5 
  の   59   53   16   11    4    3    8    4    3    1 
  を   10   96   26    3    1    7    5    0    2    6 
 とか   41   53   54    3    8    5    4   11    3   13 
 って   31   34   49    1    3    1    5    7    1    8 
 から   13   82   26    4    1    1    1    2    0    2 
  と   11   23   31    4    3    1    3    6    1    4 
 でも    1  135    4    0    0    0    3    0    1    1 


In [24]:
cfd_cleaned = {}
for w, f in most_common_joshis:
    cfd_cleaned[w] = {}
    for v, k in mujoshi_most_common_adjs:
        cfd_cleaned[w][v] = cfd[w][v]

In [25]:
table = pd.DataFrame(cfd_cleaned).fillna(0)
table = table.reindex([w for w, f in mujoshi_most_common_adjs])
table[[w for w, f in most_common_joshis]].T

,ない,いい,すごい,悪い,なかっ,なく,おいしい,多い,かわいい,すっごい
無助詞,1121,720,375,110,104,101,87,85,73,71
が,463,851,166,79,49,63,37,178,20,21
は,260,295,125,11,58,16,21,17,11,15
に,82,231,67,23,20,13,15,3,8,12
も,252,114,60,2,25,34,13,23,21,4
で,86,134,56,5,18,10,2,3,6,5
か,34,62,88,11,3,7,7,1,4,8
て,9,220,23,4,5,14,5,0,2,5
の,59,53,16,11,4,3,8,4,3,1
を,10,96,26,3,1,7,5,0,2,6


In [29]:
#calculating the total number of joshi-adj occurrences
total_occurrences = 0
for joshi in joshi_adjs.keys():
    total_occurrences += len(joshi_adjs[joshi])
total_occurrences

17587

In [30]:
#calculating the total number of mujoshi occurrences
mujoshi_occurrences = len(joshi_adjs['無助詞'])
mujoshi_occurrences

5282

In [31]:
#calculating the total number of ない (including its other forms) occurrences
nai_occurrences = 0
#calculating the total number of occurrences of the combinations of mujoshi and ない (including its other forms)
mujoshi_nai_occurrences = 0
for joshi in cfd.keys():
    for adj in cfd[joshi].keys():
        if adj == 'ない' or adj == 'なかっ' or adj == 'なく' or adj == 'なかろ' or adj == 'なけれ':
            nai_occurrences += cfd[joshi][adj]
            if joshi == '無助詞':
                mujoshi_nai_occurrences += cfd[joshi][adj]


In [33]:
nai_occurrences

3315

In [34]:
mujoshi_nai_occurrences

1330

In [35]:
mujoshi_nai_occurrences / nai_occurrences * 100

40.120663650075414

In [36]:
mujoshi_nai_occurrences / mujoshi_occurrences * 100

25.179856115107913

In [37]:
mujoshi_nai_occurrences / total_occurrences * 100

7.562404048444874